# Movie Recommendation system
This is a group project of ISSS 610 Applied Machine Learning.  

Group Member: GENG Minghong, HUANG Lu, Manish Agarwal, TAO Xinru  

Project Timeline:

Time|Jobs
-----|-----
2020-03-21|Data Cleaning
2020-03-23|Exploring
**2020-03-30**|**Presentation**

# 0. Introduction 
The data of this project can be accessed here:  
https://smu.sharepoint.com/teams/ISSS610AppliedMachineLearning/Shared%20Documents/Forms/AllItems.aspx?viewid=1b247e4e%2D6a34%2D4c70%2D8abc%2D06cee2cccbea  

# 1. Processing Data
## 1.1 Load Packages and Import the Data
The packages we used in this packages are:
- Pandas
- NumPy
- ...
- (To be fill)

In [1]:
import pandas as pd
import numpy as np

links= pd.read_csv(r'data\themoviesdataset\links.csv')


## 1.2 Data Cleaning 

In [2]:
from sklearn.metrics.pairwise import (linear_kernel, 
                                      cosine_similarity)
from ast import literal_eval
import datetime


### 1.2.1 keywords.csv

In [3]:
keywords= pd.read_csv(r'data\themoviesdataset\keywords.csv')
keywords['keywordsTr'] = keywords['keywords'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
keywordsTr=keywords.drop(['keywords'],axis=1)

In [38]:
keywordsTr.rename(columns = {'id':'tmdb_id'},inplace=True)
keywordsTr['tmdb_id'] = keywordsTr['tmdb_id'].apply(str)

keywordsTr.head()

,tmdb_id,keywordsTr
0,862,"[jealousy, toy, boy, friendship, friends, riva..."
1,8844,"[board game, disappearance, based on children'..."
2,15602,"[fishing, best friend, duringcreditsstinger, o..."
3,31357,"[based on novel, interracial relationship, sin..."
4,11862,"[baby, midlife crisis, confidence, aging, daug..."


In [39]:
keywordsTr.to_csv('keywordsTr.csv')

### 1.2.2  rating.csv
We want to transform the time stamp into readable time format.   Reference:https://blog.csdn.net/weixin_43790560/article/details/88412005

In [70]:
rating= pd.read_csv(r'data\themoviesdataset\ratings.csv')
#rating['timestampTr']=pd.to_datetime(rating['timestamp'],unit='s')
ratingTr=rating[['userId','movieId','rating','timestamp']]

In [ ]:
ratingTr.to_csv('ratingTr.csv')

In [71]:
ratingTr.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


**Then we need to decide what to keep in `timestampTr`. Day? Week? Month?**

### 1.2.3 credits.csv

In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [10]:
credits = pd.read_csv(r'data\themoviesdataset\credits.csv')
credits.head(2)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [12]:
credits['cast'] = credits['cast'].apply(literal_eval)
credits['crew'] = credits['crew'].apply(literal_eval)
credits['cast_size'] = credits['cast'].apply(lambda x: len(x))
credits['crew_size'] = credits['crew'].apply(lambda x: len(x))

Rename the `id` column.

In [13]:
credits.rename(columns = {'id':'tmdb_id'},inplace=True)
credits['tmdb_id'] = credits['tmdb_id'].apply(str)

In [14]:
credits.head(2)

,cast,crew,tmdb_id,cast_size,crew_size
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,13,106
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,26,16


In [15]:
# Define a function to get the director name
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [16]:
credits['director'] = credits['crew'].apply(get_director)

In [17]:
credits['cast'] = credits['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
credits['cast'] = credits['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [30]:
creditsTr = credits
creditsTr.head(2)

,cast,crew,tmdb_id,cast_size,crew_size,director
0,"[Tom Hanks, Tim Allen, Don Rickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,13,106,John Lasseter
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,26,16,Joe Johnston


## 1.2.4 movies_metadata.csv

In [23]:
movies_metadata= pd.read_csv(r'data\themoviesdataset\movies_metadata.csv',low_memory=False)
movies_metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In the origin dataset, 24 columns are provided. We only need 15 of them.

In [25]:
movies_metadataTr = movies_metadata[['genres','id','imdb_id','overview','popularity',
                                 'production_companies','production_countries','release_date',
                                 'revenue','runtime','spoken_languages','status',
                                 'title','vote_average','vote_count']]
print(movies_metadataTr.shape)
movies_metadataTr.head(2)

(45466, 15)


,genres,id,imdb_id,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Toy Story,7.7,5415.0
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Jumanji,6.9,2413.0


In [28]:
# Rename the column id to tmdb_id
movies_metadataTr.rename(columns = {'id':'tmdb_id'},inplace=True)

In [41]:
movies_metadataTr.head(2)

,genres,tmdb_id,imdb_id,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Toy Story,7.7,5415.0
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Jumanji,6.9,2413.0


## 1.3 Merge the data

### 1.3.1 Movie

Merge the `movies_metadataTr`, `creditsTr`, and `keywordsTr.csv`.

In [43]:
# Merge the movies_metadata.csv, credits.csv, and keywords.csv
metadataTr_creditsTr = pd.merge(movies_metadataTr, 
                   creditsTr, 
                   how = 'left', 
                   on = ['tmdb_id'])

In [45]:
metadataTr_creditsTr_keywordsTr = pd.merge(metadataTr_creditsTr, 
                                           keywordsTr, 
                                           how = 'left', 
                                           on = ['tmdb_id'])

In [46]:
metadataTr_creditsTr_keywordsTr.head(2)

,genres,tmdb_id,imdb_id,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,...,status,title,vote_average,vote_count,cast,crew,cast_size,crew_size,director,keywordsTr
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,...,Released,Toy Story,7.7,5415.0,"[Tom Hanks, Tim Allen, Don Rickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",13.0,106.0,John Lasseter,"[jealousy, toy, boy, friendship, friends, riva..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,...,Released,Jumanji,6.9,2413.0,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",26.0,16.0,Joe Johnston,"[board game, disappearance, based on children'..."


In [55]:
# Select rows where the column status == 'Released'
data_movies = metadataTr_creditsTr_keywordsTr[metadataTr_creditsTr_keywordsTr['status'] == 'Released']

In [56]:
# Fill NA by the space
data_movies = cleaned_data.fillna(' ')

In [98]:
data_movies.head(2)

,genres,tmdb_id,imdb_id,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,...,status,title,vote_average,vote_count,cast,crew,cast_size,crew_size,director,keywordsTr
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81,...,Released,Toy Story,7.7,5415.0,"[Tom Hanks, Tim Allen, Don Rickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",13,106,John Lasseter,"[jealousy, toy, boy, friendship, friends, riva..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104,...,Released,Jumanji,6.9,2413.0,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",26,16,Joe Johnston,"[board game, disappearance, based on children'..."


Genre, overview, Keywords

### 1.3.2 User

In [66]:
tags = pd.read_csv('data/ml-25m/tags.csv')

In [96]:
tags.shape

(1093360, 4)

In [94]:
tags[tags['userId']==4]

,userId,movieId,tag,timestamp
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455
5,4,44665,unreliable narrators,1573943619
6,4,115569,tense,1573943077
7,4,115713,artificial intelligence,1573942979
8,4,115713,philosophical,1573943033
9,4,115713,tense,1573943042
10,4,148426,so bad it's good,1573942965
11,4,164909,cliche,1573943721


In [95]:
ratingTr[ratingTr['userId']==4]

,userId,movieId,rating,timestamp
59,4,223,4.0,1042668576
60,4,415,4.0,1042667925
61,4,648,4.0,1042674800
62,4,1097,5.0,1042667925
63,4,1197,4.0,1042667956
...,...,...,...,...
116,4,4226,3.0,1042674412
117,4,4238,4.0,1042674819
118,4,4848,3.0,1042674425
119,4,4896,4.0,1042667845


In [74]:
data_user = pd.merge(ratingTr, tags, how = 'left', on = ['userId','movieId','timestamp'])

In [86]:
data_user['tag'].value_counts()

Series([], Name: tag, dtype: int64)